In [103]:
import sagemaker
from sagemaker.pytorch import PyTorch
import boto3
import json

In [104]:
sagemaker_session = sagemaker.Session()

bucket = 'slip-ml'

role = 'arn:aws:iam::438465160412:role/Sagemaker'

In [105]:
secret_name = "huggingface"
region_name = "us-east-1"
session = boto3.session.Session()
secretsmanager = session.client(service_name='secretsmanager', region_name=region_name)
get_secret_value_response = secretsmanager.get_secret_value(SecretId=secret_name)
secret = get_secret_value_response['SecretString']
api_key = json.loads(secret)["API_KEY"]

[05/14/25 10:32:45] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=706379;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=901088;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/botocore/credentials.py#1352\1352]8;;\

In [106]:
instance_type = 'ml.g5.12xlarge' #'ml.p3.2xlarge'

In [107]:
image_uri = sagemaker.image_uris.retrieve(framework='pytorch',
                             region=sagemaker_session.boto_region_name,
                             instance_type=instance_type,
                             image_scope='training')
print(image_uri)

[05/14/25 10:32:46] INFO     Defaulting to only available Python version: py312                   ]8;id=150512;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=773261;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagemaker/image_uris.py#610\610]8;;\

763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:2.6.0-gpu-py312


In [ ]:
estimator = PyTorch(
    entry_point="finetune_llama.py",
    role=role,
    instance_count=1,
    instance_type=instance_type,
    image_uri=image_uri,
    py_version="py310",
    source_dir="source",
    hyperparameters={
        "batch-size": 2,
        "epochs": 7,
        "lr": 3e-4,
        "project-name": "vallr-phoneme-llama",
        'bucket': f'{bucket}',
    },
    sagemaker_session=sagemaker_session,
    volume_size=100,
    environment={"HF_TOKEN": "" + api_key,}
)

In [109]:
estimator.fit({'training': f's3://{bucket}/data/vallr/train/text/',
               'test': f's3://{bucket}/data/vallr/test/text/'})

                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=352773;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=693219;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

[05/14/25 10:32:47] INFO     Creating training-job with name:                                       ]8;id=213577;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=5313;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagemaker/session.py#1042\1042]8;;\
                             pytorch-training-2025-05-14-16-32-46-224                                              

2025-05-14 16:32:47 Starting - Starting the training job...
2025-05-14 16:32:53 Pending - Training job waiting for capacity...
2025-05-14 16:33:53 Pending - Preparing the instances for training......
2025-05-14 16:34:51 Downloading - Downloading the training image.....................
2025-05-14 16:38:29 Training - Training image download completed. Training in progress......bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
CUDA compat package should be installed for NVIDIA driver smaller than 560.35.05
Current installed NVIDIA driver version is 550.163.01
Adding CUDA compat to LD_LIBRARY_PATH
/usr/local/cuda/compat:/opt/amazon/openmpi/lib:/opt/amazon/efa/lib:/lib/x86_64-linux-gnu:/usr/local/lib:/usr/local/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2025-05-14 16:39:11,770 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2025-05-14 16:39:11,807 sagemaker-training-toolkit INFO 

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 estimator.fit({'training': f's3://{bucket}/data/vallr/train/text/',                          │
│   2 │   │   │      'test': f's3://{bucket}/data/vallr/test/text/'})                              │
│   3                                                                                              │
│                                                                                                  │
│ /Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagem │
│ aker/telemetry/telemetry_logging.py:167 in wrapper                                               │
│                                                                                                  │
│   164 │   │   │   │   │   caught_ex = e                                                          │
│   165 │   │   │   │   finally:                                                                   │
│   166 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 167 │   │   │   │   │   │   raise caught_ex                                                    │
│   168 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   169 │   │   │   else:                                                                          │
│   170 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ /Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagem │
│ aker/telemetry/telemetry_logging.py:138 in wrapper                                               │
│                                                                                                  │
│   135 │   │   │   │   start_timer = perf_counter()                                               │
│   136 │   │   │   │   try:                                                                       │
│   137 │   │   │   │   │   # Call the original function                                           │
│ ❱ 138 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   139 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   140 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   141 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ /Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagem │
│ aker/workflow/pipeline_context.py:346 in wrapper                                                 │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                                                            │
│                                                            